# 1. Trabajo colaborativo

El flujo de trabajo que hasta ahora hemos visto es:

```
    > git init                  # inicializa un repositorio
    > git add <archivo>         # da a conocer a git un archivo nuevo o un cambio en un archivo
    > git status                # resume los cambios actuales
    > git commit -m "mensaje"   # saca una foto instantánea del estado actual del proyecto 
    > git log                   # muestra la bitácora del proyecto

```
Otro comando que quizá usaron en la tarea fueron:
```
    > git push                  # sube los cambios a un repositorio central (local o remoto)
```

(Por ahora ">" indicará la línea de comandos.)

Ahora veremos un modelo posible de *colaboración*, que iremos complicando poco a poco. Además profundizaremos en el uso del push. 

La situación que consideraremos es la siguiente: Alicia (*Alice*) y Beto (*Bob*) colaboran en un proyecto (el que acabamos de clonar). Ambos tienen la misma versión del código. 

**Alicia:**
Alicia edita el archivo `archivo.txt`, y hace algún cambio que le parece conveniente. Siguiendo el esquema de trabajo que describimos arriba, Alicia sube los cambios a su repositorio local con `git add` y `git commit`, y finalmente los sube al repositorio central: `git push`.

**Beto:**
Beto, por su parte y de manera independiente, hace cambios *al mismo archivo* en que trabajó Alicia. De la misma manera que lo hizo Alicia, Beto actualiza su repositorio local (`git add` y `git commit`) y los sube al repositorio que comparten con `git push`.

*Sin embargo*, como él editó el mismo archivo en el que Alicia hizo cambios, pero usando una versión atrasada que *no* incluye los cambios de Alicia, entonces `git` detecta que hubo cambios divergentes entre la versión local de Beto, en la *rama* `master`, y la del repositorio remoto `origin/master`. Esto hace que `git` no permita subir los cambios que propone Beto, hasta que Beto resuelva los *conflictos* que hayan surgido.

## Enviando los commits a otro repositorio

Vamos a tratar de producir una situación controlada en la que tengamos dicha
situación. 

Para esto vamos a crear un repositorio nuevo. Este repositorio no será de 
trabajo. Solamente tendrá la información de los archivos, y los comits. 
Para esto ejecutamos, desde un directorio de prueba (por ejemplo `/tmp` o `~/Desktop`

In [61]:
;cd ~/Desktop

/mnt/homes/home/david/Desktop


In [62]:
;git init --bare original.git

Reinitialized existing Git repository in /mnt/homes/home/david/Desktop/original.git/


Los contenidos del directorio son los contenidos del directorio ´.git´ de otros
repositorios normales. Esto es por la opción ´--bare´.   
Después clonamos este repositorio "central" a otro directorio

In [63]:
;ls -la original.git

total 2156
drwxrwxr-x  6 david david    4096 feb 21 14:58 .
drwxrwxr-- 29 david david 2170880 feb 21 14:57 ..
-rw-rw-r--  1 david david      66 feb 21 14:58 config
-rw-rw-r--  1 david david      73 feb 21 14:57 description
-rw-rw-r--  1 david david      23 feb 21 14:57 HEAD
drwxrwxr-x  2 david david    4096 feb 21 14:57 hooks
drwxrwxr-x  2 david david    4096 feb 21 14:57 info
drwxrwxr-x 32 david david    4096 feb 21 14:57 objects
drwxrwxr-x  4 david david    4096 feb 21 14:57 refs


In [64]:
;git clone original.git alice

fatal: destination path 'alice' already exists and is not an empty directory.


Pero vemos que el repositorio nuevo es normal:

In [65]:
;cd alice

/mnt/homes/home/david/Desktop/alice


In [66]:
;ls -alp 

total 2148
drwxrwxr-x  3 david david    4096 feb 21 14:57 ./
drwxrwxr-- 29 david david 2170880 feb 21 14:57 ../
-rw-rw-r--  1 david david      20 feb 21 14:57 algo.tmp
drwxrwxr-x  7 david david    4096 feb 21 14:57 .git/
-rw-rw-r--  1 david david       9 feb 21 14:57 .gitignore
-rw-rw-r--  1 david david      53 feb 21 14:57 idea_bob.txt
-rw-rw-r--  1 david david     108 feb 21 14:57 inicio.txt


Ahora creamos un archivo de prueba ahi mismo y hacemos el procedimiento usual para hacer un commit.

In [67]:
;echo "iniciamos este archivo con perros y gatos" >> inicio.txt

In [68]:
;git add . && git commit -m"Commit inicial"

[master a14fb2a] Commit inicial
 1 file changed, 1 insertion(+)


Ahora, dado que clonamos el repositorio de otro, vamos a enviar los cambios (la creación del archivo `inicio.txt`) al respositio central:

In [69]:
;git push origin master

To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


La sintaxis de este comando es `git push repository branchname`. En este caso, `repository` es la dirección o referencia de algun repositorio. Esta puede ser, por `git@github.com:git/git.git`, pero por lo generalusamos alguna que esté determinada por defecto. Por ejemplo, vamos a usar  
`origin`  
que de acuerdo a 

In [70]:
;git remote -v

origin	/mnt/homes/home/david/Desktop/original.git (fetch)
origin	/mnt/homes/home/david/Desktop/original.git (push)


es `/home/carlosp/Desktop/tmp/original.git` (en mi caso). El `branchname` es la rama. Hasta ahora, no hemos creado ramas, por lo que nos referiremos a la rama principal, que se llama `master`. Después de el primer comando `git push origin master` podemos abreviarlo como `git push`.

Ahora creamos un archivo sin importancia, y modificamos nuestro archivo importante:


In [71]:
;echo "informacion trivial" >> algo.tmp && echo "mas informacion" >> inicio.txt

In [72]:
;git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   inicio.txt

no changes added to commit (use "git add" and/or "git commit -a")


Viendo el `status` notamos que `git` sabe que hemos cambiado el archivo `inicio.txt` y hemos creado un nuevo archivo `algo.tmp`. Supongamos que por error añadimos este último archivo sin importancia:

In [73]:
;git add . && git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   inicio.txt



Entonces, el archivo `algo.tmp` se encuentra listo para que los sigamos. Sin embargo, no es lo que queremos. El comando 

In [74]:
;git rm --cached algo.tmp

fatal: pathspec 'algo.tmp' did not match any files


hace que lo borremos de lo que queremos en el commit. Si lo incluimos en el `.gitignore`, entonces vemos que ya no aparece en el status:

In [75]:
;echo "algo.tmp" >> .gitignore

In [76]:
;git status && git add .  && git commit -m"Se mejora el gitignore y se anade mas informcion a inicio.txt" && git push

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   inicio.txt

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .gitignore

[master 9bc8379] Se mejora el gitignore y se anade mas informcion a inicio.txt
 2 files changed, 2 insertions(+)


To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


## Interaccion y errores controlados

Comenzamos creando otro repositorio en el que otro personaje (Bob) va a trabajar.

In [77]:
;cd .. 

/mnt/homes/home/david/Desktop


In [78]:
;git clone original.git bob

fatal: destination path 'bob' already exists and is not an empty directory.


In [79]:
;cd bob

/mnt/homes/home/david/Desktop/bob


Para facilitar un poco la lectura de los logs, vamos a poner otro nombre para el usuario:

In [80]:
;git config user.name Bob

## Bob hace un cambio y Alice lo ve

Ahora vamos a crear un nuevo archivo, y a enviarlo.

In [81]:
;echo "esta es una idea de Bob" >> idea_bob.txt

In [82]:
;git add . && git commit -m"Nueva idea" && git push && git log

[master 335b6e2] Nueva idea
commit 335b6e284229a54ad0df71451a935bb60f7ea31b
Merge: 50bd4c3 b03a8bf
Author: Bob <davidwarro@gmail.com>
Date:   Tue Feb 21 14:58:17 2017 -0600

    Nueva idea

commit 50bd4c38521e9a69cf1e3f1cb027cddaa8857c52
Author: Bob <davidwarro@gmail.com>
Date:   Tue Feb 21 14:57:13 2017 -0600

    Primer intento de correccion

commit 985ab4f59f65fc61723652aa73848153786c979d
Merge: decb6e3 335123b
Author: Bob <davidwarro@gmail.com>
Date:   Tue Feb 21 14:57:13 2017 -0600

    Hicimos un merge manual

commit decb6e3c2d9a9a6731a9015238561ae06178f8c3
Author: Bob <davidwarro@gmail.com>
Date:   Tue Feb 21 14:57:13 2017 -0600

    Otra idea de bob

commit 335123b69c4846795edf81b3436cd8b28ea55b8d
Author: davidphysdavalos <davidwarro@gmail.com>
Date:   Tue Feb 21 14:57:13 2017 -0600

    Otra idea de alicia

commit b29537c586e9166e87ad05d468b4e76ae8715dc7
Merge: 6da6ba4 ebe3d69
Author: Bob <davidwarro@gmail.com>
Date:   Tue Feb 21 14:57:13 2017 -0600

    Merge branch 'master' 

To /mnt/homes/home/david/Desktop/original.git
   50bd4c3..335b6e2  master -> master


Ahora, veamos que es lo que Alice ve. 

In [83]:
;cd ../alice

/mnt/homes/home/david/Desktop/alice


In [84]:
;git pull

From /mnt/homes/home/david/Desktop/original
   335123b..335b6e2  master     -> origin/master


Auto-merging inicio.txt
CONFLICT (content): Merge conflict in inicio.txt
Auto-merging .gitignore
CONFLICT (content): Merge conflict in .gitignore
Automatic merge failed; fix conflicts and then commit the result.


In [85]:
;ls

algo.tmp
arreglo_bug1.txt
clases
idea_bob.txt
inicio.txt
LICENSE.md
notebooks
README.md
tareas


Efectivamente, ahora Alice tiene la nueva idea de Bob 

### Alice y Bob hacen cambios en archivos diferentes

Vamos a hacer un cambio en un archivo de Alice, y a enviarlo al repositorio. 

Posteriormente intentaremos hacer lo mismo con Bob.

In [86]:
;date >> inicio.txt && git add . && git commit -m"Se anade la fecha" && git push

[master dc022e9] Se anade la fecha


To /mnt/homes/home/david/Desktop/original.git
   335b6e2..dc022e9  master -> master


In [87]:
;cd ../bob

/mnt/homes/home/david/Desktop/bob


In [88]:
;date >> idea_bob.txt && git add . && git commit -m"Se anade otra fecha" && git push

[master 5f0035a] Se anade otra fecha
 1 file changed, 1 insertion(+)


To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


Tenemos un problema. El cambio qeu queremos hacer ha sido rechazado [rejected]

De hecho, `git` nos está diciendo como resolver el problema. Debemos traer los cambios a nuestro repositorio mediante un `pull`.

In [89]:
;git pull

Merge made by the 'recursive' strategy.
 .gitignore | 4 ++++
 inicio.txt | 6 ++++++
 2 files changed, 10 insertions(+)


From /mnt/homes/home/david/Desktop/original
   335b6e2..dc022e9  master     -> origin/master


y automaticamente se hace un "merge" que combina los cambios de las dos partes.

Sin embargo, debemos hacer push para que los cambios, que se hicieron a nivel del repositorio de Bob, queden en el repositorio central. Similarmente, actualizamos a alice.

In [90]:
;git push

To /mnt/homes/home/david/Desktop/original.git
   dc022e9..42291f8  master -> master


In [91]:
;cd ../alice

/mnt/homes/home/david/Desktop/alice


In [92]:
;git pull

Updating dc022e9..42291f8
Fast-forward
 idea_bob.txt | 1 +
 1 file changed, 1 insertion(+)


From /mnt/homes/home/david/Desktop/original
   dc022e9..42291f8  master     -> origin/master


In [93]:
;git log --graph --pretty

*   commit 42291f8c4311f31c024e8f0619309b80856617af
|\  Merge: 5f0035a dc022e9
| | Author: Bob <davidwarro@gmail.com>
| | Date:   Tue Feb 21 14:58:17 2017 -0600
| | 
| |     Merge branch 'master' of /mnt/homes/home/david/Desktop/original
| |     
| *   commit dc022e92f65d03f4dc36994b321da7755de31981
| |\  Merge: 9bc8379 335b6e2
| | | Author: davidphysdavalos <davidwarro@gmail.com>
| | | Date:   Tue Feb 21 14:58:17 2017 -0600
| | | 
| | |     Se anade la fecha
| | |    
| * | commit 9bc83797bbea512108efd3420e2c888c1c239762
| | | Author: davidphysdavalos <davidwarro@gmail.com>
| | | Date:   Tue Feb 21 14:58:17 2017 -0600
| | | 
| | |     Se mejora el gitignore y se anade mas informcion a inicio.txt
| | |    
| * | commit a14fb2a717700566ad5ff74357edda0eac6b4730
| | | Author: davidphysdavalos <davidwarro@gmail.com>
| | | Date:   Tue Feb 21 14:58:17 2017 -0600
| | | 
| | |     Commit inicial
| | |    
* | | commit 5f0035ac881ce137951b1e5c49573ce6f0cea037
| |/  Author: Bob <davidwarro@gmail

### Cuando cambiamos el mismo archivo

Esto no debería pasar. Pero en la practica, nos sucede con cierta frecuencia, especialmente editando notas. 

Vamos a hacer cambios en las dos partes (Alice y Bob) y veamos como se comporta `git` en cada una y como resolverlo. Ambas partes editarán el archivo `inicio.txt`.

In [94]:
;pwd

/mnt/homes/home/david/Desktop/alice


In [95]:
;echo "comentario de alicia" >> inicio.txt && git add . && git commit -m"Otra idea de alicia" && git push

[master 65ac135] Otra idea de alicia
 1 file changed, 1 insertion(+)


To /mnt/homes/home/david/Desktop/original.git
   42291f8..65ac135  master -> master


In [96]:
;cd ../bob

/mnt/homes/home/david/Desktop/bob


In [97]:
;echo "comentario de bob" >> inicio.txt && git add . && git commit -m"Otra idea de bob"

[master 61def39] Otra idea de bob
 1 file changed, 1 insertion(+)


In [98]:
;git push

To /mnt/homes/home/david/Desktop/original.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to '/mnt/homes/home/david/Desktop/original.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


Naturalmente, los cambios de nuevo son rechazados. Intentamos hacer un pull, pero en esta ocasión tenemos un mensaje de error un poco mas severo. 

In [99]:
;git pull

Auto-merging inicio.txt
CONFLICT (content): Merge conflict in inicio.txt
Automatic merge failed; fix conflicts and then commit the result.


From /mnt/homes/home/david/Desktop/original
   42291f8..65ac135  master     -> origin/master


Este error indica que tenemos qeu resolver el conflicto en forma manuala. Si nuestro archivo es código puro, como en este caso, no hay problema. Si es un archivo un poco mas complicado, típicamente toca escoger alguna de las versiones. En nuestro caso, podemos ver los dos contenidos del archivo:

In [100]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
mar feb 21 14:57:13 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
<<<<<<< HEAD
iniciamos este archivo con perros y gatos
mas informacion
>>>>>>> 335123b69c4846795edf81b3436cd8b28ea55b8d
propuesta para arreglo de bug 1
>>>>>>> 335b6e284229a54ad0df71451a935bb60f7ea31b
mar feb 21 14:58:17 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
>>>>>>> 65ac135344c22564aff30109c668a6913ebd301d


Podemos editarlo manualmente para que ahora se vea asi:

In [101]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
mar feb 21 14:57:13 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
<<<<<<< HEAD
iniciamos este archivo con perros y gatos
mas informacion
>>>>>>> 335123b69c4846795edf81b3436cd8b28ea55b8d
propuesta para arreglo de bug 1
>>>>>>> 335b6e284229a54ad0df71451a935bb60f7ea31b
mar feb 21 14:58:17 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
>>>>>>> 65ac135344c22564aff30109c668a6913ebd301d


Y ya podemos hacer un commit y su posterior push.

In [102]:
;git add . && git commit -m"Hicimos un merge manual" && git push && git status

[master 56d99d9] Hicimos un merge manual
On branch master
Your branch is up-to-date with 'origin/master'.

nothing to commit, working directory clean


To /mnt/homes/home/david/Desktop/original.git
   65ac135..56d99d9  master -> master


# 2. Trabajando en ramas

El concepto de una *rama* ("branch") en git provee una forma sencilla y eficiente de trabajar en nuevas ideas, o de colaborar en un proyecto común, evitando romper cosas que a priori ya funcionan.

Para empezar, listemos las ramas existentes de un proyecto (por ejemplo, en el directorio `Alicia/`):
```
    > git branch
```
o usando
```
    > git branch -v
```
que brinda además el hash del último commit. Lo que esto indica es que existe únicamente la rama `master`, que es la rama que se crea por default (y en algún sentido es la principal), y el asterisco indica que estamos trabajando en esa rama.

Para crear una nueva rama, ejecutamos:
```
    > git branch <nombre_rama>
```
donde `<nombre_rama>` es el nombre de la rama, que es más o menos arbitrario y flexible. Un 
ejemplo es: `bugfix1`.

Después de ejecutar alguna de estas instrucciones, `git branch -v` nos informa que *ambas* ramas, `master` y `bugfix1` existen, ambas están en el (mismo) último commit, y el asterisco indica que estamos en la rama `master` aún.

Para cambiarnos de rama, ejecutamos:
```
    > git checkout <nombre_rama>
```
Nuevamente, existe un atajo para crear y cambiarnos de rama de un golpe: `git checkout -b <nombre_rama>`.

Ahora, la linea de status cobra mas sentido. 

In [103]:
;git checkout -b bugfix1 && git status

fatal: A branch named 'bugfix1' already exists.


Ahora podemos crear un archivo nuevo, y modificar uno existente. 

In [104]:
;echo "propuesta para arreglo de bug 1" >> inicio.txt && echo "nuevo codigo qeu lo corrige" >> arreglo_bug1.txt

In [105]:
;git add . && git commit -m"Primer intento de correccion"

[master 231ac5a] Primer intento de correccion
 2 files changed, 2 insertions(+)


In [106]:
;ls

arreglo_bug1.txt
clases
idea_bob.txt
inicio.txt
LICENSE.md
notebooks
README.md
tareas


In [107]:
;cat inicio.txt

iniciamos este archivo con perros y gatos
mas informacion
mar feb 21 14:57:13 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
<<<<<<< HEAD
iniciamos este archivo con perros y gatos
mas informacion
>>>>>>> 335123b69c4846795edf81b3436cd8b28ea55b8d
propuesta para arreglo de bug 1
>>>>>>> 335b6e284229a54ad0df71451a935bb60f7ea31b
mar feb 21 14:58:17 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
>>>>>>> 65ac135344c22564aff30109c668a6913ebd301d
propuesta para arreglo de bug 1


In [108]:
;git checkout master

Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)


Already on 'master'


In [109]:
;ls

arreglo_bug1.txt
clases
idea_bob.txt
inicio.txt
LICENSE.md
notebooks
README.md
tareas


In [110]:
;cat inicio.txt 

iniciamos este archivo con perros y gatos
mas informacion
mar feb 21 14:57:13 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
<<<<<<< HEAD
iniciamos este archivo con perros y gatos
mas informacion
>>>>>>> 335123b69c4846795edf81b3436cd8b28ea55b8d
propuesta para arreglo de bug 1
>>>>>>> 335b6e284229a54ad0df71451a935bb60f7ea31b
mar feb 21 14:58:17 CST 2017
<<<<<<< HEAD
comentario de bob
comentario de alicia
>>>>>>> 65ac135344c22564aff30109c668a6913ebd301d
propuesta para arreglo de bug 1


El punto importante hasta el momento es que la historia de los dos branches (locales) ha divergido, y *ambas* historias están en ambas ramas.

Supongamos ahora que ya están satisfechos con los cambios que han hecho, después de muchas pruebas exhaustivas y otras fallidas (tal vez en otras ramas). Ahora queremos poner estos cambios en la rama `master`. Para esto, primero nos cambiamos a `master`, que es la rama a donde queremos pasar los cambios, y después hacemos un `merge`, o sea, fundimos las dos historias:
```
    > git checkout master   
    > git merge <nombre_rama>
```
En nuestro caso, podemos simplemente hacer un merge, de la misma manera que haciamos con otros repositorios. En este caso, lo podemos hacer directamente. 

In [111]:
;git checkout master && git merge bugfix1

Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)
Already up-to-date.


Already on 'master'


De nuevo, un `git log` puede resultar particularmente util, si hemos hecho cambios en diferentes branches.

#  Apendice 1: Facilitar el flujo de trabajo con GitHub

Para evitar que GitHub te esté pidiendo tu usuario y contraseña todo el tiempo, es necesario usar *claves de SSH* (SSH keys). En Linux y Mac, el procedimiento es como sigue. NB: **No** hacer esto desde una máquina/cuenta pública.

Utiliza el comando `ssh-keygen` para generar claves nuevas. Te pedirá que pongas una clave ("passphrase"); esta clave tendrás que ponerla sólo una vez por sesión.

In [112]:
;ssh-keygen

Generating public/private rsa key pair.


Enter file in which to save the key (/home/david/.ssh/id_rsa): 

Esto generará claves en el directorio escondido `~/.ssh` en tu directorio hogar:

In [113]:
;ls ~/.ssh

authorized_keys
config
config~
id_rsa
id_rsa.pub
known_hosts
known_hosts.old
sabe


Ahora, copia la clave pública; esto se puede hacer a mano (copiando el *contenido*  del archivo `id_rsa.pub`), o usando un programa. E.g. en Mac, puedes usar `pbcopy` para copiar el contenido de un archivo al clipboard:

In [114]:
;pbcopy < ~/.ssh/id_rsa.pub

/bin/bash: pbcopy: command not found


Ahora, hay que dar de alta las claves en GitHub:

- Ve a la página de tu cuenta en GitHub
- Escoge `Settings` (arriba, del lado derecho)
- Escoge `SSH keys`
- Escoge `Add SSH key`
- Pega lo que copiaste

Ya deberías poder hacer transacciones con GitHub sin que te pida tu usuario cada vez.


Una vez más, **no** hagas esto en un máquina o cuenta pública.

# Apendice 2 Trabajar con un fork

Normalmente hacemos un **fork** de un repositorio de interés en GitHub, es decir, una copia del repositorio en tu propia cuenta de GitHub.

Al hacer `git clone ...` de tu fork, `git` provee un remote (es decir, un nombre para un repositorio remoto) llamado `origin`, que apunta a tu fork. Esto lo podemos ver con

In [6]:
;cd Desktop

/mnt/homes/home/david/Desktop


In [7]:
;git clone https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git

Cloning into 'MetodosNumericosAvanzados'...


In [9]:
;cd MetodosNumericosAvanzados

/mnt/homes/home/david/Desktop/MetodosNumericosAvanzados


In [10]:
; git remote -v

origin	https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git (fetch)
origin	https://github.com/davidphysdavalos/MetodosNumericosAvanzados.git (push)


Vemos que `origin` apunta al fork del repositorio `MetodosNumericosAvanzados` en mi cuenta de GitHub (con usuario `dpsanders`).

Sin embargo, para mantener actualizado nuestro fork con respecto al repositorio original, debemos darle a conocer a `git` que también existe dicho repositorio. Si hacemos

In [11]:
;git help remote

GIT-REMOTE(1)                     Git Manual                     GIT-REMOTE(1)



NAME
       git-remote - manage set of tracked repositories

SYNOPSIS
       git remote [-v | --verbose]
       git remote add [-t <branch>] [-m <master>] [-f] [--[no-]tags] [--mirror=<fetch|push>] <name> <url>
       git remote rename <old> <new>
       git remote remove <name>
       git remote set-head <name> (-a | --auto | -d | --delete | <branch>)
       git remote set-branches [--add] <name> <branch>...
       git remote set-url [--push] <name> <newurl> [<oldurl>]
       git remote set-url --add [--push] <name> <newurl>
       git remote set-url --delete [--push] <name> <url>
       git remote [-v | --verbose] show [-n] <name>...
       git remote prune [-n | --dry-run] <name>...
       git remote [-v | --verbose] update [-p | --prune] [(<group> | <remote>)...]


DESCRIPTION
       Manage the set of repositories ("remotes") whose branches you track.

OPTIONS
       -v, --verbose
           Be a litt

vemos que hay un subcomando `add` de remote. Así que hacemos

In [12]:
; git remote add upstream https://github.com/dpsanders/MetodosNumericosAvanzados

El nombre usual que se le asigna al repositorio original es `upstream`.

Ahora podemos actualizar nuestro repositorio *local* con 

In [13]:
; git pull upstream master

Already up-to-date.


From https://github.com/dpsanders/MetodosNumericosAvanzados
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> upstream/master


(el cual jala la rama `master` del repositorio apuntado por `upstream`).

Ahora al hacer

In [14]:
; git push

Everything up-to-date


empuja los cambios a `origin`, o sea, a nuestro propio fork.

* __Ejercicio:__ Hacer un fork del repositorio de la clase en su cuenta de github. Hacer un git clone de su fork y agregar un archivo con el formato que deben llevar las tareas, donde el contenido sea el nombre de su cerveza favorita.

Recordemos el formato de la tarea: T(numero)(nombre).extension

# Para la siguiente iteracion

* Revisar que el git config funciona donde lo voy a hacer. 
* Poner los dibujitos explicitamente en el notebook

